In [1]:
# .txt-->.xml
# ! /usr/bin/python
# -*- coding:UTF-8 -*-
import os
import cv2
from tqdm import tqdm
import numpy as np

In [2]:
# 按文本生成xml 支持中文路径，支持多图片格式

def txt_to_xml(txt_path, img_path, xml_path, label_dict,dataset_name):
    if not os.path.exists(xml_path): #判断所在目录下是否有该文件名的文件夹
        os.mkdir(xml_path)
    
    # 2.找到txt标签文件夹
    files = os.listdir(txt_path)

    # 3.遍历文件夹
    for name in tqdm(files):
        # 许多人文件夹里有该文件，默认的也删不掉，那就直接pass
        if name == "desktop.ini":
            continue

        # 4.打开txt
        txtFile = open(txt_path + name)
        # 读取所有内容
        txtList = txtFile.readlines()
        # 读取图片名称
        img_name = name.split(".txt")[0]
        # print(img_name)
        if os.path.exists(img_path + img_name + ".jpg"):
            # pic = cv2.imread(img_path + img_name + ".jpg")
            pic = cv2.imdecode(np.fromfile(os.path.join(img_path, img_name + ".jpg"), dtype=np.uint8), -1)
        elif os.path.exists(img_path + img_name + ".JPG"):
            # pic = cv2.imread(img_path + img_name + ".JPG")
            pic = cv2.imdecode(np.fromfile(os.path.join(img_path, img_name + ".JPG"), dtype=np.uint8), -1)
            
        elif os.path.exists(img_path + img_name + ".png"):
            # pic = cv2.imread(img_path + img_name + ".png")
            pic = cv2.imdecode(np.fromfile(os.path.join(img_path, img_name + ".png"), dtype=np.uint8), -1)
        elif os.path.exists(img_path + img_name + ".jpeg"):
            # pic = cv2.imread(img_path + img_name + ".png")
            pic = cv2.imdecode(np.fromfile(os.path.join(img_path, img_name + ".jpeg"), dtype=np.uint8), -1)
                
        elif os.path.exists(img_path + img_name + ".bmp"):
            # pic = cv2.imread(img_path + img_name + ".bmp")
            pic = cv2.imdecode(np.fromfile(os.path.join(img_path, img_name + ".bmp"), dtype=np.uint8), -1)
            
        else:
            print(img_name,'图片格式不支持或图片不存在')
        # 获取图像大小信息
        try:
            Pheight, Pwidth, Pdepth = pic.shape
        except:
            print(img_name,'图片读取失败')
            continue

        xml_file_list = []
        
        # 5.遍历txt文件中每行内容
        for row in txtList:
            # 按' '分割txt的一行的内容
            oneline = row.strip().split(" ")
            # 遇到的是一张新图片
            if img_name not in xml_file_list:
            # if img_name != pre_img_name:
                # 6.新建xml文件
                xml_file = open((xml_path + img_name + '.xml'), 'w')
                xml_file.write('<annotation>\n')
                xml_file.write('    <folder>'+ dataset_name+ '</folder>\n')
                xml_file.write('    <filename>' + img_name + '.jpg' + '</filename>\n')
                xml_file.write('    <path>' + img_path + '</path>\n')
                xml_file.write('    <source>\n')
                xml_file.write('        <database>orgaquant</database>\n')
                xml_file.write('        <annotation>organoids</annotation>\n')
                xml_file.write('    </source>\n')
                xml_file.write('    <size>\n')
                xml_file.write('        <width>' + str(Pwidth) + '</width>\n')
                xml_file.write('        <height>' + str(Pheight) + '</height>\n')
                xml_file.write('        <depth>' + str(Pdepth) + '</depth>\n')
                xml_file.write('    </size>\n')
                
                xml_file.write('    <object>\n')
                xml_file.write('        <name>' + label_dict[oneline[0]] + '</name>\n')
                xml_file.write('        <bndbox>\n')
                xml_file.write('            <xmin>' + str(
                    int(((float(oneline[1])) * Pwidth + 1) - (float(oneline[3])) * 0.5 * Pwidth)) + '</xmin>\n')
                xml_file.write('                <ymin>' + str(
                    int(((float(oneline[2])) * Pheight + 1) - (float(oneline[4])) * 0.5 * Pheight)) + '</ymin>\n')
                xml_file.write('                <xmax>' + str(
                    int(((float(oneline[1])) * Pwidth + 1) + (float(oneline[3])) * 0.5 * Pwidth)) + '</xmax>\n')
                xml_file.write('                <ymax>' + str(
                    int(((float(oneline[2])) * Pheight + 1) + (float(oneline[4])) * 0.5 * Pheight)) + '</ymax>\n')
                xml_file.write('        </bndbox>\n')
                xml_file.write('    </object>\n')
                xml_file.close()

                xml_file_list.append(img_name)
            else:
                # 7.同一张图片，只需要追加写入object
                xml_file = open((xml_path + img_name + '.xml'), 'a')
                xml_file.write('    <object>\n')
                xml_file.write('        <name>' + label_dict[oneline[0]] + '</name>\n')
                xml_file.write('        <bndbox>\n')
                xml_file.write('            <xmin>' + str(
                    int(((float(oneline[1])) * Pwidth + 1) - (float(oneline[3])) * 0.5 * Pwidth)) + '</xmin>\n')
                xml_file.write('            <ymin>' + str(
                    int(((float(oneline[2])) * Pheight + 1) - (float(oneline[4])) * 0.5 * Pheight)) + '</ymin>\n')
                xml_file.write('            <xmax>' + str(
                    int(((float(oneline[1])) * Pwidth + 1) + (float(oneline[3])) * 0.5 * Pwidth)) + '</xmax>\n')
                xml_file.write('            <ymax>' + str(
                    int(((float(oneline[2])) * Pheight + 1) + (float(oneline[4])) * 0.5 * Pheight)) + '</ymax>\n')
                xml_file.write('        </bndbox>\n')
                xml_file.write('    </object>\n')
                xml_file.close()
                
        # 8.读完txt文件最后写入</annotation>
        xml_file1 = open((xml_path + img_name + '.xml'), 'a')
        xml_file1.write('</annotation>')
        xml_file1.close()
    print("Done !")


In [3]:
# 修改成自己的文件夹 注意文件夹最后要加上/
dataset_name = 'zhatuche_jzlaji'
txt_path = "E:/puzhou_train/jietu/predict/labels/"
img_path = "E:/puzhou_train/jietu/jzlaji/"
xml_path = "E:/puzhou_train/jietu/jzlaji_xml/"
# 1.字典对标签中的类别进行转换
label_dict = {'0':"wjj", '1':"ztc", '2':"ttc",'3': 'jzlaji'}
            
txt_to_xml(txt_path,img_path,xml_path,label_dict,dataset_name)

100%|████████████████████████████████████████████████████████████████████████████████| 152/152 [00:06<00:00, 23.05it/s]

Done !


In [34]:
from xml.dom.minidom import Document
import os
import cv2


# 按格式生成xml
def makexml(picPath, txtPath, xmlPath):  # txt所在文件夹路径，xml文件保存路径，图片所在文件夹路径
    """此函数用于将yolo格式txt标注文件转换为voc格式xml标注文件
    """
    if not os.path.exists(xmlPath):
        os.mkdir(xmlPath)
    dic = {'0': "fire",  # 创建字典用来对类型进行转换
           '1': "1",  # 此处的字典要与自己的classes.txt文件中的类对应，且顺序要一致
           '3' :'person'
           }
    files = os.listdir(txtPath)
    for i, name in enumerate(files):
        xmlBuilder = Document()
        annotation = xmlBuilder.createElement("annotation")  # 创建annotation标签
        xmlBuilder.appendChild(annotation)
        txtFile = open(txtPath + name)
        txtList = txtFile.readlines()
        img = cv2.imread(picPath + name[0:-4] + ".png")
        Pheight, Pwidth, Pdepth = img.shape

        folder = xmlBuilder.createElement("folder")  # folder标签
        foldercontent = xmlBuilder.createTextNode("driving_annotation_dataset")
        folder.appendChild(foldercontent)
        annotation.appendChild(folder)  # folder标签结束

        filename = xmlBuilder.createElement("filename")  # filename标签
        filenamecontent = xmlBuilder.createTextNode(name[0:-4] + ".png")
        filename.appendChild(filenamecontent)
        annotation.appendChild(filename)  # filename标签结束

        size = xmlBuilder.createElement("size")  # size标签
        width = xmlBuilder.createElement("width")  # size子标签width
        widthcontent = xmlBuilder.createTextNode(str(Pwidth))
        width.appendChild(widthcontent)
        size.appendChild(width)  # size子标签width结束

        height = xmlBuilder.createElement("height")  # size子标签height
        heightcontent = xmlBuilder.createTextNode(str(Pheight))
        height.appendChild(heightcontent)
        size.appendChild(height)  # size子标签height结束

        depth = xmlBuilder.createElement("depth")  # size子标签depth
        depthcontent = xmlBuilder.createTextNode(str(Pdepth))
        depth.appendChild(depthcontent)
        size.appendChild(depth)  # size子标签depth结束

        annotation.appendChild(size)  # size标签结束

        for j in txtList:
            oneline = j.strip().split(" ")
            object = xmlBuilder.createElement("object")  # object 标签
            picname = xmlBuilder.createElement("name")  # name标签
            namecontent = xmlBuilder.createTextNode(dic[oneline[0]])
            picname.appendChild(namecontent)
            object.appendChild(picname)  # name标签结束

            pose = xmlBuilder.createElement("pose")  # pose标签
            posecontent = xmlBuilder.createTextNode("Unspecified")
            pose.appendChild(posecontent)
            object.appendChild(pose)  # pose标签结束

            truncated = xmlBuilder.createElement("truncated")  # truncated标签
            truncatedContent = xmlBuilder.createTextNode("0")
            truncated.appendChild(truncatedContent)
            object.appendChild(truncated)  # truncated标签结束

            difficult = xmlBuilder.createElement("difficult")  # difficult标签
            difficultcontent = xmlBuilder.createTextNode("0")
            difficult.appendChild(difficultcontent)
            object.appendChild(difficult)  # difficult标签结束

            bndbox = xmlBuilder.createElement("bndbox")  # bndbox标签
            xmin = xmlBuilder.createElement("xmin")  # xmin标签
            mathData = int(((float(oneline[1])) * Pwidth + 1) - (float(oneline[3])) * 0.5 * Pwidth)
            xminContent = xmlBuilder.createTextNode(str(mathData))
            xmin.appendChild(xminContent)
            bndbox.appendChild(xmin)  # xmin标签结束

            ymin = xmlBuilder.createElement("ymin")  # ymin标签
            mathData = int(((float(oneline[2])) * Pheight + 1) - (float(oneline[4])) * 0.5 * Pheight)
            yminContent = xmlBuilder.createTextNode(str(mathData))
            ymin.appendChild(yminContent)
            bndbox.appendChild(ymin)  # ymin标签结束

            xmax = xmlBuilder.createElement("xmax")  # xmax标签
            mathData = int(((float(oneline[1])) * Pwidth + 1) + (float(oneline[3])) * 0.5 * Pwidth)
            xmaxContent = xmlBuilder.createTextNode(str(mathData))
            xmax.appendChild(xmaxContent)
            bndbox.appendChild(xmax)  # xmax标签结束

            ymax = xmlBuilder.createElement("ymax")  # ymax标签
            mathData = int(((float(oneline[2])) * Pheight + 1) + (float(oneline[4])) * 0.5 * Pheight)
            ymaxContent = xmlBuilder.createTextNode(str(mathData))
            ymax.appendChild(ymaxContent)
            bndbox.appendChild(ymax)  # ymax标签结束

            object.appendChild(bndbox)  # bndbox标签结束

            annotation.appendChild(object)  # object标签结束

        with open(xmlPath + name[0:-4] + ".xml", 'w') as f:
            xmlBuilder.writexml(f, indent='\t', newl='\n', addindent='\t', encoding='utf-8')
        # f.close()


if __name__ == "__main__":
    picPath = "E:/test/images/"  # 图片所在文件夹路径，后面的/一定要带上
    txtPath = "E:/test/labels/"  # txt所在文件夹路径，后面的/一定要带上
    xmlPath = "E:/test/xml100/"  # xml文件保存路径，后面的/一定要带上
    makexml(picPath, txtPath, xmlPath)

